In [42]:
from bs4 import BeautifulSoup as bs
import requests
from collections import OrderedDict
import pandas as pd
import numpy as np
import re


In [43]:
url = 'https://www.imdb.com/list/ls507232408/?sort=release_date,asc&st_dt=&mode=detail&page=1'
page_source = requests.get(url)
soup = bs(page_source.content, 'lxml')

In [44]:
movies = []
movie_characters = []
movie_lengths = OrderedDict()

In [45]:
for movie in soup.find_all('div', class_='lister-item mode-detail'):
		# Fetching title of the movie
		movie_title = movie.find('div', class_='lister-item-content').h3.a.text
		if movie_title == "¿Qué pasaría si...?":
			continue
		# Fetching length of the movie
		movie_length = movie.find('div', class_='lister-item-content').find('span', class_='runtime').text
		movies.append(movie_title)
		movie_lengths[movie_title] = movie_length.split(' ')[0]
		# Fetching characters in the movie
		characters = movie.find('div', class_='list-description').p.text
		movie_characters.append(characters.split('\n'))

In [46]:
data_dict = OrderedDict()
# Inputing data into the dictionary
for i in range(len(movies)):
	data_dict[movies[i]] = movie_characters[i]

In [47]:
character_movie_time = {}
# Parsing through data scraped for each movie
for movie in data_dict.keys():
	# Parsing through each character in in the movie
	for item in data_dict[movie]:
		character = item.split('<')[0].strip()
		time = item.split('<')[1].split('>')[0].strip()
		# Adding time for each character to for each movie to the index dict
		if movie in character_movie_time.keys():
			character_movie_time[movie][character] = time
		else:
			character_movie_time[movie] = {character: time}

In [48]:
char_id_index = pd.read_csv('characters.csv')
# Creating aa dict containing character IDs for each character
names = {old:new for (old, new) in zip(char_id_index['Character Name'], char_id_index['Character ID'])}
# Converting the data dict into a pandas data frame for easy handling
char_movie_matrix = pd.DataFrame(character_movie_time)
char_movie_matrix

,Iron Man,El increíble Hulk,Iron Man 2,Thor,Capitán América: El primer vengador,Marvel de un vistazo: El consultor,El caso único de Marvel: Algo divertido ocurrió de camino al martillo de Thor,Los Vengadores,Marvel extendido: Artículo 47,Iron Man 3,...,Ojo de Halcón,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas
Tony Stark / Iron Man,77:15,:30,61:15,NaN,NaN,:30,NaN,31:45,NaN,62:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pepper Potts,23:15,NaN,17:30,NaN,NaN,NaN,NaN,4:30,NaN,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Obadiah Stane / Iron Monger,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Professor Ho Yinsen,10:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Lt. Col. James ""Rhodey"" Rhodes",8:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mining Tribe Elder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:30,NaN
U.S. Secretary of State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
Kevin Bacon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:30
Bzermiktoklok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:45


In [49]:
# Renaming characters with their character IDs
char_movie_matrix.rename(index=names, inplace=True)
char_movie_matrix

,Iron Man,El increíble Hulk,Iron Man 2,Thor,Capitán América: El primer vengador,Marvel de un vistazo: El consultor,El caso único de Marvel: Algo divertido ocurrió de camino al martillo de Thor,Los Vengadores,Marvel extendido: Artículo 47,Iron Man 3,...,Ojo de Halcón,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas
Tony Stark,77:15,:30,61:15,NaN,NaN,:30,NaN,31:45,NaN,62:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pepper Potts,23:15,NaN,17:30,NaN,NaN,NaN,NaN,4:30,NaN,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Obadiah Stane,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Professor Ho Yinsen,10:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
James Rhodes,8:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mining Tribe Elder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:30,NaN
U.S. Secretary of State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
Kevin Bacon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:30
Bzermiktoklok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:45


In [50]:
def combine_rows(matrix):
	"""Method that combines the duplicate rows that arose due to replacing character IDs"""
	# List of characters
	characters = list(set([character for character in matrix.index]))
	# Iterating over every character
	for character in characters:
		# Creating a mini data frame for each character
		df = matrix.loc[character]
		# If the character row is present just once, iterating over every column would be iterating over every movie
		if len(list(df.index)) != len(matrix.columns):
			# Iterating over every movie for each character
			for movie in matrix.columns:
				# Initializing update value to NaN
				value = np.nan
				li = []
				# Creating a list of values for each movie
				for i in range(len(df.index)):
					li.append(str(df.iloc[i][movie]))
				# Updating the right value
				for item in li:
					if str(item) != 'nan':
						value = item
				# Updating value to the location
				matrix.loc[character, movie] = value
	# If there are n duplicates, n rows would have been created with same values; deleting the duplicates
	matrix = matrix.drop_duplicates()
	return matrix

In [51]:
# Combine rows with same index
char_movie_matrix = combine_rows(char_movie_matrix)
char_movie_matrix

,Iron Man,El increíble Hulk,Iron Man 2,Thor,Capitán América: El primer vengador,Marvel de un vistazo: El consultor,El caso único de Marvel: Algo divertido ocurrió de camino al martillo de Thor,Los Vengadores,Marvel extendido: Artículo 47,Iron Man 3,...,Ojo de Halcón,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas
Tony Stark,77:15,:30,61:15,NaN,NaN,:30,NaN,31:45,NaN,62:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pepper Potts,23:15,NaN,17:30,NaN,NaN,NaN,NaN,4:30,NaN,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Obadiah Stane,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Professor Ho Yinsen,10:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
James Rhodes,8:15,NaN,18:45,NaN,NaN,NaN,NaN,NaN,NaN,9:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zawavari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:30,NaN
U.S. Secretary of State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
Kevin Bacon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:30
Bzermiktoklok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:45


In [52]:
char_movie_matrix.index.tolist()

['Tony Stark',
 'Pepper Potts',
 'Obadiah Stane',
 'Professor Ho Yinsen',
 'James Rhodes',
 'Raza',
 'Phil Coulson',
 'Christine Everhart',
 'Abu Bakaar',
 'Happy',
 'William Ginter Riva',
 'Nick Fury',
 'J.A.R.V.I.S.',
 'Hulk',
 'Dr. Betty Ross',
 'Emil Blonsky / The Abomination',
 'General Thaddeus Ross',
 'Samuel Sterns',
 'Dr. Leonard Samson',
 'Roger Harrington',
 'Ivan Vanko',
 'Justin Hammer',
 'Black Widow',
 'Senator Stern',
 'Howard Stark',
 'Anton Vanko',
 'Peter Parker',
 'Thor',
 'Loki',
 'Jane Foster',
 'Erik Selvig',
 'Odin',
 'Darcy Lewis',
 'Sif',
 'Volstagg',
 'Fandral',
 'Hogun',
 'Heimdall',
 'Laufey',
 'Frigga',
 'Agent Jasper Sitwell',
 'Hawkeye',
 'Steve Rogers',
 'Peggy Carter',
 'Red Skull',
 'Col. Chester Phillips',
 'Dr. Abraham Erskine',
 'Arnim Zola',
 'Bucky',
 'Heinz Kruger',
 'Sgt. Timothy "Dum Dum" Dugan',
 'Pvt. Gabe Jones',
 'Pvt. Jim Morita',
 'Gen. Thaddeus Ross',
 'Maria Hill',
 'The Other',
 'Gen. Georgi Luchkov',
 'Councilwoman Malick',
 'Council

In [53]:
def convert_time_to_mins(matrix):
	"""Method that converts time of the format mm:ss or :ss or mm to minutes in float"""
	# Iterating over each character
	for character in matrix.index:
		# Iterating over each movie
		for movie in matrix.columns:
			# Current value of the cell in the matrix
			value = matrix.loc[character][movie]
			# Iterating over non NaN values only
			if str(value) != 'nan':
				# To convert strings of the format mm:ss
				if re.match(r'\d+:\d+', value):
					matrix.loc[character][movie] = float(value.split(':')[0]) + float(value.split(':')[1])/60
				# To convert strings of the format :ss
				elif re.match(r'^:\d+', value):
					matrix.loc[character][movie] = float(value.split(':')[1])/60
				# To convert strings of the format mm
				elif re.match(r'^\d+[^:]', value):
					matrix.loc[character][movie] = float(value)
				# Other formats assigned as NaN
				else:
					matrix.loc[character][movie] = np.nan
			# NaN values reassigned as NaN
			else:
				matrix.loc[character][movie] = np.nan
	return matrix

In [54]:
convert_time_to_mins(char_movie_matrix)

,Iron Man,El increíble Hulk,Iron Man 2,Thor,Capitán América: El primer vengador,Marvel de un vistazo: El consultor,El caso único de Marvel: Algo divertido ocurrió de camino al martillo de Thor,Los Vengadores,Marvel extendido: Artículo 47,Iron Man 3,...,Ojo de Halcón,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas
Tony Stark,77.25,0.5,61.25,NaN,NaN,0.5,NaN,31.75,NaN,62.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pepper Potts,23.25,NaN,17.5,NaN,NaN,NaN,NaN,4.5,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Obadiah Stane,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Professor Ho Yinsen,10.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
James Rhodes,8.25,NaN,18.75,NaN,NaN,NaN,NaN,NaN,NaN,9.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zawavari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN
U.S. Secretary of State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kevin Bacon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.5
Bzermiktoklok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.75


In [55]:
char_movie_matrix['Total de tiempo en pantalla'] = char_movie_matrix.loc[:,:].sum(axis=1, skipna=True)

C:\Users\lache\AppData\Local\Temp\ipykernel_10348\2284275073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  char_movie_matrix['Total de tiempo en pantalla'] = char_movie_matrix.loc[:,:].sum(axis=1, skipna=True)


In [56]:
char_movie_matrix = char_movie_matrix.sort_values(by='Total de tiempo en pantalla', ascending=False)

In [57]:
char_movie_matrix 

,Iron Man,El increíble Hulk,Iron Man 2,Thor,Capitán América: El primer vengador,Marvel de un vistazo: El consultor,El caso único de Marvel: Algo divertido ocurrió de camino al martillo de Thor,Los Vengadores,Marvel extendido: Artículo 47,Iron Man 3,...,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas,Total de tiempo en pantalla
Tony Stark,77.25,0.5,61.25,NaN,NaN,0.5,NaN,31.75,NaN,62.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.25
Thor,NaN,NaN,NaN,43.25,NaN,NaN,NaN,18.0,NaN,NaN,...,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,262.0
Peter Parker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,243.25
Steve Rogers,NaN,NaN,NaN,NaN,56.5,NaN,NaN,28.25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.75
Scarlett Witch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
Ann Marie Hoag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
Actor Loki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
Ben,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
